## Zip to S3

This notebook automates the (re)build, install, zip, and upload to S3 required to update a AWS Layer.

In [3]:
from my_utils.os_utils import subprocess_execute, zipdir

In [4]:
# list of packages to build
packages_todo = [
    # list of tuples: ('package_name','from_pip')
    #('pandas',True),
    ('my_utils',False),
    ('my_dev',False),    
]
package_base_dir = 'C:\\Users\\amaguire\\Documents\\GitHub\\my-python-packages'
s3_path = 's3://andrewm4894-python-packages/latest/'
s3_region = 'us-west-2'
s3_bucket = s3_path.split('s3://')[1].split('/')[0]
s3_url = s3_path.replace('s3://','https://').replace(s3_bucket,f'{s3_bucket}.s3-{s3_region}.amazonaws.com')

In [14]:
# build and install locally each package
for package_name, from_pip in packages_todo:
    cmd = ''
    # run pip install within the python dir
    if from_pip:
        # if from pip then just use name
        cmd = cmd + f'pip install --upgrade --force-reinstall "{package_name}" -t {package_base_dir}\\python\\'
    else:
        # cd into the package dir
        cmd = f'cd {package_base_dir}\\{package_name}'
        # run the build commands to create local egg file to be zipped up into an AWS layer
        cmd = cmd + f' && python setup.py bdist_egg'
        # cd into the 'python' dir which is to be zipped up and sent to AWS
        cmd = cmd + f' && cd {package_base_dir}\\python'
        # otherwise point to local dir
        cmd = cmd + f' && pip install --upgrade "{package_base_dir}\\{package_name}" -t .'
    # kick off big command to do the stuff
    result = subprocess_execute(cmd)
    # look at results
    print(result)

... cmd to execute:
cd C:\Users\amaguire\Documents\GitHub\my-python-packages\my_utils && python setup.py bdist_egg && cd C:\Users\amaguire\Documents\GitHub\my-python-packages\python && pip install --upgrade "C:\Users\amaguire\Documents\GitHub\my-python-packages\my_utils" -t .
... result:
running bdist_egg
running egg_info
writing my_utils.egg-info\PKG-INFO
writing dependency_links to my_utils.egg-info\dependency_links.txt
writing top-level names to my_utils.egg-info\top_level.txt
reading manifest file 'my_utils.egg-info\SOURCES.txt'
writing manifest file 'my_utils.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\my_utils
copying build\lib\my_utils\os_utils.py -> build\bdist.win-amd64\egg\my_utils
copying build\lib\my_utils\__init__.py -> build\bdist.win-amd64\egg\my_utils
byte-compiling build\bdist.win-amd64\egg\my_utils\os_utils.py to os_utils.cpython-36

In [5]:
# make zip
zipdir('python/','python.zip')
# upload zip to s3 using aws cli
cmd = f'aws s3 cp --region {s3_region} {package_base_dir}\\python.zip {s3_path}'
print(subprocess_execute(cmd))
print(f'... zip should be updated at: {s3_url}python.zip ...')

... cmd to execute:
aws s3 cp --region us-west-2 C:\Users\amaguire\Documents\GitHub\my-python-packages\python.zip s3://andrewm4894-python-packages/latest/
... result:
upload: .\python.zip to s3://andrewm4894-python-packages/latest/python.zip

... zip should be updated at: https://andrewm4894-python-packages.s3-us-west-2.amazonaws.com/latest/python.zip ...
